In [3]:
import urllib.parse
import requests
import pandas as pd
import datetime
import io
import re
import progressbar
from pythainlp.util.date import thai_strptime

In [ ]:
startYear = 1996
today = datetime.date.today()
currentYear = today.year
all_dfs = []

In [ ]:
def buddhistYear(year):
    return year+543

In [ ]:
# dlType = "interestCSV" # interestCSV or interestXLS
# dateFrom = "01/1/2565" # dd/mm/YYYY * Year in Buddhist calendar. Eg. 13/1/2565
# dateTo = "31/12/2565" # dd/mm/YYYY * Year in Buddhist calendar. Eg. 13/1/2565
# bankValue = "002" # bankId
# bankName = "กรุงเทพ" # bankName
# rdoTypeStr = "1" # 0 for deposit or 1 for loan

def getBOT_URL(dateFrom,dateTo,bankValue,bankName,rdoTypeStr):
    payload = {
        "dlType" : dlType,
        "dateFrom" : dateFrom,
        "dateTo" : dateTo,
        "bankValue" : bankValue,
        "bankName" : bankName,
        "rdoTypeStr" : rdoTypeStr
    }
    url = "https://www.bot.or.th/thai/statistics/_layouts/application/interest_rate/DownloadFile.aspx?"    
    return url+urllib.parse.urlencode(payload)

In [ ]:
bank_df = pd.read_csv("bank.csv",header=0,names=["bankValue","bankName"],dtype={"bankValue":str,"bankName":str})

In [ ]:
# overallBar = progressbar.ProgressBar()

In [ ]:
for index,row in bank_df.iterrows():
    bankValue = row.bankValue
    bankName = row.bankName
    bankBar = progressbar.ProgressBar()
    print("Bank : {}".format(bankName))
    for year in bankBar(range(startYear,currentYear+1)):
        dateFrom = "01/01/{}".format(buddhistYear(year))
        dateTo = "31/12/{}".format(buddhistYear(year))

        if(year==startYear):
            dateFrom="31/01/{}".format(buddhistYear(year))
        elif(year==currentYear):
            date = today.day
            month = today.month
            dateTo = "{}/{}/{}".format(date,month,buddhistYear(year))

        dlType = "interestCSV"
        rdoTypeStr = "1" 

        url = getBOT_URL(dateFrom,dateTo,bankValue,bankName,rdoTypeStr)
        res = requests.get(url)
        if(re.search("ไม่พบข้อมูลสำหรับสร้างไฟล์",res.text)):
            print("Skip {} for year {}".format(bankName,year))
        else:
            df = pd.read_csv(io.StringIO(res.text),sep="|",encoding="iso8859_11",skiprows=[0,1])
            df["bank"] = bankName
            all_dfs.append(df)

In [ ]:
consolidated_df = pd.concat(all_dfs)

In [ ]:
# print(consolidated_df)

In [30]:
consolidated_df.to_csv("./output/loan-interest-rates.csv",index=False)

In [6]:
consolidated_df = pd.read_csv("./output/loan-interest-rates.csv",index_col=False)

In [28]:
print(consolidated_df)

        Unnamed: 0         วันที่     MOR    MLR    MRR  สูงสุด  ผิดนัด  \
0                0   31 ม.ค. 2539  14.000  13.75  14.25   16.25    18.0   
1                1   29 ก.พ. 2539  14.000  13.75  14.25   16.25    18.0   
2                2  31 มี.ค. 2539  13.750  13.75  14.25   16.25    18.0   
3                3  30 เม.ย. 2539  13.750  13.75  14.25   16.25    18.0   
4                4   31 พ.ค. 2539  13.750  13.75  13.75   15.75    18.0   
...            ...            ...     ...    ...    ...     ...     ...   
292536           3   06 ม.ค. 2566   6.875   6.00    NaN   15.00    15.0   
292537           4   09 ม.ค. 2566   6.875   6.00    NaN   15.00    15.0   
292538           5   10 ม.ค. 2566   6.875   6.00    NaN   15.00    15.0   
292539           6   11 ม.ค. 2566   6.875   6.00    NaN   15.00    15.0   
292540           7   12 ม.ค. 2566   6.875   6.00    NaN   15.00    15.0   

       บัตรเครดิต                           bank                      date  
0               -     

In [26]:
def convert_date(txt):
    try:
        return thai_strptime(txt,'%d %B %Y')
    except:
        return ''

In [27]:
consolidated_df['date'] = consolidated_df['วันที่'].apply(convert_date)

/tmp/ipykernel_75511/2239779024.py:1: FutureWarning: Inferring datetime64[ns, Asia/Bangkok] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns, Asia/Bangkok])
  consolidated_df['date'] = consolidated_df['วันที่'].apply(convert_date)


In [31]:
thai_strptime("29 ก.พ. 2539","%d %B %Y")

IndexError: list index out of range